# Recommedation of medicine by giving a disease name 

In [1]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc

import os
import string
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999


In [2]:
df = pd.read_csv("Drug.csv")

In [3]:
df.head()

,Condition,Drug,Indication,Type,Reviews,Effective,EaseOfUse,Satisfaction,Information
0,Acute Bacterial Sinusitis,Levofloxacin,On Label,RX,994 Reviews,2.52,3.01,1.84,\r\n\t\t\t\t\tLevofloxacin is used to treat a ...
1,Acute Bacterial Sinusitis,Levofloxacin,On Label,RX,994 Reviews,2.52,3.01,1.84,\r\n\t\t\t\t\tLevofloxacin is used to treat a ...
2,Acute Bacterial Sinusitis,Moxifloxacin,On Label,RX,755 Reviews,2.78,3.00,2.08,\r\n\t\t\t\t\t This is a generic drug. The ave...
3,Acute Bacterial Sinusitis,Azithromycin,On Label,RX,584 Reviews,3.21,4.01,2.57,\r\n\t\t\t\t\tAzithromycin is an antibiotic (m...
4,Acute Bacterial Sinusitis,Azithromycin,On Label,RX,584 Reviews,3.21,4.01,2.57,\r\n\t\t\t\t\tAzithromycin is an antibiotic (m...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Condition     2219 non-null   object 
 1   Drug          2219 non-null   object 
 2   Indication    2219 non-null   object 
 3   Type          2219 non-null   object 
 4   Reviews       2219 non-null   object 
 5   Effective     2219 non-null   float64
 6   EaseOfUse     2219 non-null   float64
 7   Satisfaction  2219 non-null   float64
 8   Information   2219 non-null   object 
dtypes: float64(3), object(6)
memory usage: 156.1+ KB


In [5]:
df=df.drop_duplicates()

In [6]:
# replace the empty cells with 'Unknown'
df['Indication'].replace('\r\n', 'Unknown', inplace=True)

# replace the empty cells with 'Unknown'
df['Type'].replace('\r\n', 'Unknown', inplace=True)

In [7]:
df_grouped = df.groupby('Indication').agg({'Satisfaction': 'mean'}).reset_index()

fig = px.bar(df_grouped, x='Indication', y='Satisfaction', 
             title='Average Satisfaction Rating by Indication')
fig.show()

<IPython.core.display.Javascript object>

In [8]:
fig = px.box(df, x='Indication', y='Effective', 
             title='Distribution of Drug Effectiveness by Indication')
fig.show()

<IPython.core.display.Javascript object>

In [9]:
fig = px.box(df, x='Type', y='Effective', color='Type', title='Drug Effectiveness by Type')
fig.show()

<IPython.core.display.Javascript object>

In [10]:
fig = px.bar(df, x='Indication', y='Satisfaction', color='Indication', title='Drug Satisfaction by Indication')
fig.show()

<IPython.core.display.Javascript object>

In [11]:
import plotly.graph_objs as go

df_agg = df.groupby(['Type']).agg({'EaseOfUse': 'mean', 'Effective': 'mean'}).reset_index()

fig = go.Figure(data=[
    go.Bar(name='Ease of Use', x=df_agg['Type'], y=df_agg['EaseOfUse']),
    go.Bar(name='Effectiveness', x=df_agg['Type'], y=df_agg['Effective'])
])

fig.update_layout(title='Drug Ease of Use and Effectiveness by Type', barmode='stack')
fig.show()

In [16]:
condition = 'Acute Bacterial Sinusitis' 

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the features to be used in the model
features = ['EaseOfUse', 'Effective', 'Satisfaction', 'Information']

# Create a new column with the combined features
df['combined_features'] = df[features].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Filter the dataset by the input condition
df_condition = df[df['Condition'] == condition]

# Define the TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the combined features
tfidf_matrix = tfidf.fit_transform(df_condition['combined_features'])

# Calculate the cosine similarity between the documents
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Get the indices of the top 5 drugs with the highest cosine similarity
drug_indices = np.argsort(cosine_sim[-1])[:-6:-1]

In [22]:
print('Recommended drugs for {}:'.format(condition))
for idx in drug_indices[1:]:
    print('- {}'.format(df_condition.iloc[idx]['Drug']))

Recommended drugs for Acute Bacterial Sinusitis:
- Erythromycin Ethylsuccinate
- Erythromycin Ethylsuccinate
- Erythromycin Ethylsuccinate
- Erythromycin Ethylsuccinate


In [15]:
import pickle
pickle.dump(df.to_dict(),open('drug_dict.pkl','wb'))
pickle.dump(cosine_sim,open('drug_similarity.pkl','wb'))

# Recommendation of other drugs by giving drug name  

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [87]:
df1 = pd.read_excel("Medicine_description (1).xlsx")

In [88]:
df1.isnull().sum()

Drug_Name       0
Reason          0
Description    14
dtype: int64

In [89]:
df1.dropna(inplace=True)

In [90]:
df1.duplicated().sum()

0

In [91]:
df1['Description'].apply(lambda x:x.split())

0                      [Mild, to, moderate, acne, (spots)]
1        [A, RET, 0.025%, is, a, prescription, medicine...
2        [It, is, used, to, treat, acne, vulgaris, in, ...
3        [It, is, used, to, treat, acne, vulgaris, in, ...
4        [treat, the, most, severe, form, of, acne, (no...
                               ...                        
22476                         [used, for, treating, warts]
22477                 [used, to, soften, the, skin, cells]
22478                                   [used, for, scars]
22479                                  [used, for, wounds]
22480    [used, to, treat, and, remove, raised, warts, ...
Name: Description, Length: 22467, dtype: object

In [92]:
df1['Reason'] = df1['Reason'].apply(lambda x:x.split())
df1['Description'] = df1['Description'].apply(lambda x:x.split())

In [93]:
df1['Description'] = df1['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [94]:
df1['Description'] = df1['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [95]:
df1['tags'] = df1['Description'] + df1['Reason'] 

In [96]:
new_df = df1[['Drug_Name','tags']]

In [97]:
new_df['tags'].apply(lambda x:" ".join(x))

0                       Mild to moderate acne (spots) Acne
1        A RET 0.025% is a prescription medicine that i...
2        It is used to treat acne vulgaris in people 12...
3        It is used to treat acne vulgaris in people 12...
4        treat the most severe form of acne (nodular ac...
                               ...                        
22476                        used for treating warts Wound
22477                  used to soften the skin cells Wound
22478                                 used for scars Wound
22479                                used for wounds Wound
22480    used to treat and remove raised warts (usually...
Name: tags, Length: 22467, dtype: object

In [98]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [99]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [100]:
import nltk

In [101]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [102]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',max_features=5000)

In [103]:
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)  

In [104]:
new_df['tags'] = new_df['tags'].apply(stem)

In [105]:
cv.fit_transform(new_df['tags']).toarray().shape

(22467, 795)

In [106]:

vectors = cv.fit_transform(new_df['tags']).toarray()

In [107]:
cv.get_feature_names()

['025',
 '12',
 '16',
 '18',
 'abdomin',
 'abl',
 'ach',
 'acid',
 'acn',
 'acne',
 'acquir',
 'action',
 'activ',
 'acut',
 'acute',
 'adequ',
 'adhd',
 'adjunct',
 'adolesc',
 'adult',
 'adults',
 'affect',
 'ag',
 'age',
 'aids',
 'allerg',
 'allergen',
 'allergi',
 'allow',
 'alon',
 'alzheim',
 'alzheimer',
 'amoebiasi',
 'anaemia',
 'anal',
 'angina',
 'angl',
 'ani',
 'ankl',
 'ankylos',
 'anorexia',
 'anoth',
 'anti',
 'antioxid',
 'antipsychot',
 'antiretrovir',
 'anxieti',
 'anxiou',
 'anxious',
 'apnoea',
 'appear',
 'appetit',
 'appetite',
 'appli',
 'appropri',
 'area',
 'arrhythmia',
 'arrhythmiasi',
 'arteri',
 'arthralgia',
 'arthriti',
 'associ',
 'atherothrombot',
 'athlete',
 'atop',
 'atrial',
 'attack',
 'awak',
 'b1',
 'b2',
 'b3',
 'b5',
 'b6',
 'babi',
 'backache',
 'bacteri',
 'bacteria',
 'balanc',
 'balanitis',
 'bandag',
 'becom',
 'behaviour',
 'beliefs',
 'benefit',
 'beta',
 'biliari',
 'biotin',
 'bite',
 'blackhead',
 'blackheads',
 'bleaches',
 'bleed'

In [108]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [109]:
def recommend(medicine):
    medicine_index = new_df[new_df['Drug_Name'] == medicine].index[0]
    distances = similarity[medicine_index]
    medicines_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in medicines_list:
        print(new_df.iloc[i[0]].Drug_Name)
    

In [110]:
recommend("Paracetamol 125mg Syrup 60mlParacetamol 500mg Tablet 10'S")

Freenac SP Tablet 10'S
GABAPRIME PLUS Gel(Topical) 30gm
Gesic Capsule 10'S
Glenpar Spas 20/325mg Tablet 10'S
Gransaid P Tablet 10'S


In [111]:
import pickle
pickle.dump(new_df.to_dict(),open('medicine_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))